<a href="https://colab.research.google.com/github/r-kovalch/acter-ner/blob/main/notebooks/acter-gliner-large-v2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/r-kovalch/acter-ner

Cloning into 'acter-ner'...
remote: Enumerating objects: 546, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 546 (delta 25), reused 6 (delta 5), pack-reused 491 (from 1)
Receiving objects: 100% (546/546), 4.58 MiB | 9.17 MiB/s, done.
Resolving deltas: 100% (351/351), done.


In [ ]:
!git clone https://github.com/AylaRT/ACTER

Cloning into 'ACTER'...
remote: Enumerating objects: 5448, done.
remote: Counting objects: 100% (5448/5448), done.
remote: Compressing objects: 100% (3304/3304), done.
remote: Total 5448 (delta 2684), reused 4893 (delta 2132), pack-reused 0 (from 0)
Receiving objects: 100% (5448/5448), 11.90 MiB | 12.00 MiB/s, done.
Resolving deltas: 100% (2684/2684), done.


In [ ]:
%cd /content/ACTER

/content/ACTER


In [ ]:
%cd /content/acter-ner/term_extractor

/content/acter-ner/term_extractor


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/ner/datasets/train_full.tsv train.tsv && \
  cp /content/drive/MyDrive/ner/datasets/val_full.tsv val.tsv

In [ ]:
import json
from pathlib import Path

def tsv_iob_to_json(tsv_path, json_path):
    sentences = []
    tokens, tags = [], []
    with open(tsv_path, encoding="utf-8") as f:
        for line in f:
            line = line.rstrip("\n")
            if not line.strip():
                if tokens:
                    sentences.append((tokens, tags))
                    tokens, tags = [], []
                continue
            tok, tag = line.split("\t")[:2]
            tokens.append(tok)
            tags.append(tag)
    if tokens:
        sentences.append((tokens, tags))
    json_records = []
    for toks, tgs in sentences:
        ner_spans = []
        i = 0
        while i < len(tgs):
            tag = tgs[i]
            if tag.startswith("B-"):
                label = tag[2:]
                start = i
                i += 1
                while i < len(tgs) and tgs[i] == f"I-{label}":
                    i += 1
                ner_spans.append([start, i-1, label])
            else:
                i += 1
        json_records.append({"tokenized_text": toks, "ner": ner_spans})
    with open(json_path, "w", encoding="utf-8") as out_file:
        json.dump(json_records, out_file, ensure_ascii=False, indent=2)

tsv_iob_to_json("train.tsv", "train.json")
tsv_iob_to_json("val.tsv", "val.json")


In [ ]:
!pip install -U "gliner>=0.2.19" "transformers>=4.51.0" \
               datasets accelerate evaluate seqeval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
  

In [ ]:
import torch, json, random
from datasets import load_dataset
from gliner import GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing import GLiNERDataset, WordsSplitter
from gliner.data_processing.collator import DataCollatorWithPadding
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
import torch, re, random, itertools, json
from collections import Counter
from transformers import EarlyStoppingCallback, TrainingArguments, Trainer

In [ ]:
train_path = "train.json"
test_path = "val.json"

with open(train_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)

print('Train dataset size:', len(train_data))

with open(test_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

print('Test dataset size:', len(test_data))

random.shuffle(train_data)
print('Train data is shuffled...')

train_dataset = train_data[:int(len(train_data) * 0.2)]
test_dataset = test_data

print('Dataset is prepared')

Train dataset size: 11343
Test dataset size: 2833
Train data is shuffled...
Dataset is prepared


In [ ]:
MODEL_NAME = 'gliner_large-v2'
EXPERIMENT_NAME = 'rerank_max'

model = GLiNER.from_pretrained(f"urchade/{MODEL_NAME}")
model.to("cuda")

data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [15]:
num_steps = 500
log_steps = 50
batch_size = 24
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

GLINER_THRESHOLD = 0.35
labels=sorted(["TERM"])


_PRF_RE = re.compile(
    r"P:\s*([0-9.]+)%\s*R:\s*([0-9.]+)%\s*F1:\s*([0-9.]+)%", re.I
)

def compute_metrics(_eval_pred):
    """
    Handles   model.evaluate() -> (output_str, f1)
    where output_str looks like:  "P: 78.42%\\tR: 71.95%\\tF1: 75.03%\\n"
    Returns ents_p / ents_r / ents_f (in %) and 'score' = F1 (0-1).
    """
    out_str, f1 = model.evaluate(                 # <- your Evaluator method
        test_dataset,
        threshold=GLINER_THRESHOLD,
        entity_types=labels
    )

    # -------- extract P and R from the string -----------------------------
    m = _PRF_RE.search(out_str)
    if not m:
        raise ValueError(f"Cannot parse PRF from: {out_str!r}")
    p, r = (float(m.group(1)), float(m.group(2)))   # already %
    # f1 returned by evaluate() is 0-1, convert to %
    f1_pct = f1 * 100

    return {
        "ents_p": p,
        "ents_r": r,
        "ents_f": f1_pct,
        "score":  f1,           # 0-1 scalar for best-model tracking
    }

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-5,
    weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    eval_strategy="steps",
    save_steps = log_steps,
    logging_steps = log_steps,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    remove_unused_columns=False,
    # label_names=["ner_labels"],
    fp16=False,
    dataloader_drop_last=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.can_return_loss = True

trainer.train()

<ipython-input-15-900069450>:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Step,Training Loss,Validation Loss,Ents P,Ents R,Ents F,Score
50,203.558700,11.720887,53.220000,70.230000,60.552363,0.605524
100,100.791200,7.662903,66.750000,85.670000,75.034460,0.750345
150,76.695500,6.140581,80.160000,84.850000,82.442885,0.824429
200,57.464000,4.949275,80.000000,91.320000,85.284454,0.852845
250,59.479100,4.887450,78.870000,94.190000,85.849295,0.858493
300,73.467700,8.687624,76.380000,77.360000,76.865054,0.768651
350,63.901700,4.570418,86.510000,93.820000,90.015989,0.900160
400,40.332200,3.307621,88.660000,93.410000,90.974664,0.909747
450,35.928000,2.991061,90.180000,93.060000,91.598323,0.915983
500,37.955400,3.987184,88.490000,94.830000,91.554205,0.915542


Step,Training Loss,Validation Loss,Ents P,Ents R,Ents F,Score
50,203.558700,11.720887,53.220000,70.230000,60.552363,0.605524
100,100.791200,7.662903,66.750000,85.670000,75.034460,0.750345
150,76.695500,6.140581,80.160000,84.850000,82.442885,0.824429
200,57.464000,4.949275,80.000000,91.320000,85.284454,0.852845
250,59.479100,4.887450,78.870000,94.190000,85.849295,0.858493
300,73.467700,8.687624,76.380000,77.360000,76.865054,0.768651
350,63.901700,4.570418,86.510000,93.820000,90.015989,0.900160
400,40.332200,3.307621,88.660000,93.410000,90.974664,0.909747
450,35.928000,2.991061,90.180000,93.060000,91.598323,0.915983
500,37.955400,3.987184,88.490000,94.830000,91.554205,0.915542


KeyboardInterrupt: 

In [ ]:
# # path to the best-loss checkpoint
# best_path = "/content/drive/MyDrive/ucu/ner/gliner_acter_en_large_v2_5_ft/checkpoint-284"
# print("Best checkpoint:", best_path)
# best_model = GLiNER.from_pretrained(best_path).to("cuda")
# out_str, f1 = best_model.evaluate(
#     dev_py,
#     threshold=0.35,
#     entity_types=labels,
#     batch_size=1         # adjust until it fits
# )
# print(out_str)          # P: 65.06%	R: 88.52%	F1: 75.00%


In [ ]:
# from google.colab import runtime
# runtime.unassign()